In [14]:
import os
import sys
sys.path.append('..')
from collections import defaultdict
import pandas as pd
import data_utils

MRT = [2,3]

all_results = []

for mrt in MRT:
    results = defaultdict(list)
    results_dirs = [
            f'v2_MRT{mrt}_SimpleModels_every_valid_day',
            f'v2_MRT{mrt}_Kalman_every_valid_day',
            f'v2_MRT{mrt}_every_valid_day',
            f'v2_MRT{mrt}_Transformer_every_valid_day'
            ]    
    
    for rel_rd, model in zip(results_dirs, ['SimpleModels', 'Kalman', 'PLRNN', 'Transformer']):
        rd = data_utils.join_ordinal_bptt_path('results', rel_rd)
        for dirpath, dirnames, filenames in os.walk(rd):
            if os.path.split(dirpath)[1].isnumeric():
                results['path'].append(dirpath)
                results['model'].append(model)
                results['is_runpath'].append(True)
                models = sum([f.endswith('.pt') for f in filenames])
                results['n_models'].append(models)
            elif any([f.endswith('.pt') for f in filenames]):
                results['path'].append(dirpath)
                results['model'].append(model)
                results['is_runpath'].append(False)
                models = sum([f.endswith('.pt') for f in filenames])
                results['n_models'].append(models)

    results = pd.DataFrame(results)
    results['MRT'] = mrt
    all_results.append(results)

all_results = pd.concat(all_results, axis=0)


Is there any runpath with more than 1 model?

In [15]:
len(all_results.loc[all_results.loc[all_results['is_runpath'], 'n_models'] > 1, 'path'])

0

Do the additional models come from a previous, cancelled run? A piece of evidence is the run number

In [16]:
all_results.loc[all_results.loc[all_results['is_runpath'], 'n_models'] > 1, 'path'].apply(lambda s: os.path.split(s)[1]).unique()

array([], dtype=object)

Which MRTs do the additional models come from?

In [17]:
all_results.loc[all_results.loc[all_results['is_runpath'], 'n_models'] > 1, 'MRT'].unique()

array([], dtype=int64)

Is there any model outside of a runpath?

In [18]:
all_results.loc[~all_results['is_runpath'], 'path'].unique()

array([], dtype=object)

Proportion of failed runs

In [19]:
all_results.groupby(['MRT', 'model'])[['n_models', 'is_runpath']].agg(lambda x: (x>0).sum())

n_models  is_runpath
MRT model                             
2   Kalman            3591        4110
    PLRNN             4080        4110
    SimpleModels      1632        1644
    Transformer       4110        4110
3   Kalman            3441        4110
    PLRNN             4110        4110
    SimpleModels      1644        1644
    Transformer       4110        4110

Which SimpleModel runs failed?

In [20]:
for p in all_results.loc[(all_results['model']=='SimpleModels') & (all_results['n_models']==0), 'path'].sort_values():
    print(p)

/home/janik.fechtelpeter/Documents/ordinal-bptt/results/v2_MRT2_SimpleModels_every_valid_day/data_12600_52.csv_participant_52_date_174.0_model_InputsRegression/001
/home/janik.fechtelpeter/Documents/ordinal-bptt/results/v2_MRT2_SimpleModels_every_valid_day/data_12600_52.csv_participant_52_date_174.0_model_MeanPredictor/001
/home/janik.fechtelpeter/Documents/ordinal-bptt/results/v2_MRT2_SimpleModels_every_valid_day/data_12600_52.csv_participant_52_date_174.0_model_MovingAverage(1)/001
/home/janik.fechtelpeter/Documents/ordinal-bptt/results/v2_MRT2_SimpleModels_every_valid_day/data_12600_52.csv_participant_52_date_174.0_model_VAR1/001
/home/janik.fechtelpeter/Documents/ordinal-bptt/results/v2_MRT2_SimpleModels_every_valid_day/data_12600_52.csv_participant_52_date_180.0_model_InputsRegression/001
/home/janik.fechtelpeter/Documents/ordinal-bptt/results/v2_MRT2_SimpleModels_every_valid_day/data_12600_52.csv_participant_52_date_180.0_model_MeanPredictor/001
/home/janik.fechtelpeter/Documents

Did PLRNN and Kalman also fail in these conditions? How many models did they produce?

In [21]:
MRT_participant_date = ((2, 52, 174.0), (2, 52, 180.0), (2, 52, 186.0))
for mrt, participant, date in MRT_participant_date:
    regexp = f'participant_{participant}_date_{date}'
    print(all_results.loc[(all_results['path'].str.contains(regexp)) & (all_results['model']=='PLRNN'), 'n_models'].sum())
    print(all_results.loc[(all_results['path'].str.contains(regexp)) & (all_results['model']=='KalmanFilter'), 'n_models'].sum())

0
0
0
0
0
0
